## Build the dataframe
Import packages

In [48]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [49]:
import pandas as pd
import numpy as np

In [50]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Use bs4 and pandas to download the table and create the dataframe

In [51]:
# url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# The wikipedia article was updated, so I am using the archived page.
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"
html_data = requests.get(url).text

In [52]:
soup = BeautifulSoup(html_data,"html5lib")

In [53]:
soup.find('tbody').find_all('tr')[1]

<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>

In [54]:
df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in soup.find('tbody').find_all('tr'):
    col = row.find_all('td')
    try:
        postal = col[0].text
        bor = col[1].text
        nei = col[2].text
        df = df.append({"PostalCode":postal, "Borough":bor, "Neighborhood":nei},ignore_index=True)
    except IndexError:
        continue

In [55]:
df.replace('\n','',regex=True,inplace=True) # remove the \n
df=df[df['Borough']!='Not assigned'] # remove the not assigned codes
df['Neighborhood'][df['Neighborhood']=='Not assigned']=df['Borough'][df['Neighborhood']=='Not assigned'] # if a neighborhood is not assigned, give the name of the borough

In [56]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [57]:
df.shape

(103, 3)

## Import coordinates
Below is the code for geocoder, but it is not able to retrieve the data from google

In [58]:
import geocoder

#for i in range(df['PostalCode'].shape[0]):
#    coords = None
#    
#    while(coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(df.iloc[i,0]))
#        coords = g.latlng
#        
#    df['Latitude'].iloc[i] = coords[0]
#    df['Longitude'].iloc[i] = coords[1]

I will load the coordinates from the csv instead.

In [59]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
df = df.merge(df_coords, left_on = 'PostalCode', right_on = 'Postal Code')[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [88]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Explore and Cluster neighborhoods
I will use only the more central neighborhoods. Since I don't know anything about Toronto, I will judge by the coordinates. I will calculate the mean and standard deviation of latitude and longitude.

In [62]:
lat_std, long_std = df.std(axis=0,numeric_only=True) # standard deviation

In [63]:
lat_mean, long_mean = df.mean(axis=0,numeric_only=True) # mean

Take only the postal codes that are 1 standard deviation away from the mean (in both coordinates)

In [64]:
central=df[(df['Latitude'].between(lat_mean-lat_std,lat_mean+lat_std)) & (df['Longitude'].between(long_mean-long_std,long_mean+long_std))]
central

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
10,M6B,North York,Glencairn,43.709577,-79.445073
13,M3C,North York,Don Mills,43.725900,-79.340923


Create the function that will run EXPLORE for a given location.

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



#### Define Credentials and Version

In [66]:
CLIENT_ID = 'J0BZW5GEJSIH4SS0YGQE1XZ3WZZ0ELEX1BCDBYJ1EH4XQSGS' # your Foursquare ID
CLIENT_SECRET = '3EAW2F3JBH3HEJARJFHXLRGNKPZRQIJKFHITVNY0MXHFPUZ2' # your Foursquare Secret
ACCESS_TOKEN = 'HVGIQR4OEMBZUULL2CURS4VWKRWTHPDZEHOKV1XCTNVIQA5U' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value


Run the function for every neighborhood.

In [67]:
toronto_venues = getNearbyVenues(names=central['Neighborhood'],
                                   latitudes=central['Latitude'],
                                   longitudes=central['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Central Bay Street
Christie
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Dufferin, Dovercourt Village
East Toronto, Broadview North (Old East York)
Downsview
The Danforth West, Riverdale
India Bazaar, The Beaches West
North Park, Maple Leaf Park, Upwood Park
Studio District
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
York Mills West
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
University of Toronto, Harbord
Moore Park, Summerhill East
Kensi

In [68]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
Caledonia-Fairbanks,4,4,4,4,4,4
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,73,73,73,73,73,73
Davisville,35,35,35,35,35,35
Davisville North,7,7,7,7,7,7


In [69]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [70]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25
4,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.016393,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.027397,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,...,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


Create a dataframe displaying the top 10 venues for each neighborhood

In [108]:
# Function that sorts the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Diner,Restaurant,Mobile Phone Shop
1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Grocery Store,Thai Restaurant,Locksmith,Breakfast Spot,Liquor Store,Restaurant,Sushi Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Brewery,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Pizza Place
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Modern European Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Thai Restaurant,Burger Joint,Japanese Restaurant,Salad Place,Spa


#### Cluster the neighborhoods into 4 clusters

In [109]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3])

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [111]:
toronto_merged = central

# merge toronto_grouped with central to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(0, inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int32')
toronto_merged


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Bus Stop,Food & Drink Shop,Yoga Studio,Modern European Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Other Great Outdoors,Park,Mediterranean Restaurant,Men's Store,Metro Station
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Yoga Studio,Shoe Store,Chocolate Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Accessories Store,Clothing Store,Carpet Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mobile Phone Shop,Movie Theater,Modern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Gym,Sandwich Place,Fried Chicken Joint,Burrito Place,Smoothie Shop,Café,Mexican Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Café,Flea Market,Bank,Intersection,Gastropub,Pharmacy,Hotel
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Thai Restaurant,Lingerie Store
10,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Bakery,Park,Japanese Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
13,M3C,North York,Don Mills,43.725900,-79.340923,0,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket


Visualize the clusters

In [112]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Examine clusters

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Other Great Outdoors,Park,Mediterranean Restaurant,Men's Store,Metro Station
2,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Yoga Studio,Shoe Store,Chocolate Shop
3,North York,0,Accessories Store,Clothing Store,Carpet Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mobile Phone Shop,Movie Theater,Modern European Restaurant
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Diner,Gym,Sandwich Place,Fried Chicken Joint,Burrito Place,Smoothie Shop,Café,Mexican Restaurant
7,North York,0,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket
8,East York,0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Café,Flea Market,Bank,Intersection,Gastropub,Pharmacy,Hotel
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Thai Restaurant,Lingerie Store
10,North York,0,Pizza Place,Bakery,Park,Japanese Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
13,North York,0,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket
14,East York,0,Park,Curling Ice,Skating Rink,Beer Store,Athletics & Sports,Yoga Studio,Modern European Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store


In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Yoga Studio,Modern European Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


In [115]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Bus Stop,Food & Drink Shop,Yoga Studio,Modern European Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
40,North York,2,Airport,Park,Bus Stop,Yoga Studio,Movie Theater,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,3,Park,Women's Store,Pool,Modern European Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
66,North York,3,Park,Convenience Store,Yoga Studio,Modern European Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
91,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Modern European Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station


### Name the clusters accordingly

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 'Cluster Labels'] = 'Free time'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 'Cluster Labels'] = 'Transportation'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 'Cluster Labels'] = 'Garden'
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 'Cluster Labels'] = 'Food and Coffee'

In [122]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,Transportation,Park,Bus Stop,Food & Drink Shop,Yoga Studio,Modern European Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,Food and Coffee,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Other Great Outdoors,Park,Mediterranean Restaurant,Men's Store,Metro Station
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Food and Coffee,Coffee Shop,Bakery,Pub,Park,Café,Theater,Breakfast Spot,Yoga Studio,Shoe Store,Chocolate Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Food and Coffee,Accessories Store,Clothing Store,Carpet Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Coffee Shop,Mobile Phone Shop,Movie Theater,Modern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Food and Coffee,Coffee Shop,Sushi Restaurant,Diner,Gym,Sandwich Place,Fried Chicken Joint,Burrito Place,Smoothie Shop,Café,Mexican Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,Food and Coffee,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Food and Coffee,Pizza Place,Gym / Fitness Center,Athletics & Sports,Café,Flea Market,Bank,Intersection,Gastropub,Pharmacy,Hotel
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Food and Coffee,Clothing Store,Coffee Shop,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Thai Restaurant,Lingerie Store
10,M6B,North York,Glencairn,43.709577,-79.445073,Food and Coffee,Pizza Place,Bakery,Park,Japanese Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
13,M3C,North York,Don Mills,43.725900,-79.340923,Food and Coffee,Gym,Coffee Shop,Beer Store,Restaurant,Sporting Goods Shop,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Supermarket
